In [3]:
#Problem: Forecast demand for a jeans brand for the coming 6 months.

#Data: We have monthly sales quantity available for 2 years (from May 2019 to May 2021) in the CSV file.

#Import all required Packages
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from pandas import DataFrame
import numpy as np
from datetime import timedelta
#import calender


,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8


In [12]:
# Read in data and display first 5 rows
features = pd.read_csv('train.csv')
features.head()

,date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
0,2010-03-10 18:00:00,13.1,46.0,0.7578,1387.2,1087.8,1056.0,1742.8,1293.4,2.5,12.0,167.7
1,2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2,2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117.0,2.2,9.2,127.1
3,2010-03-10 21:00:00,11.0,62.4,0.7867,1321.0,929.0,1102.9,1536.5,1263.2,2.2,9.7,177.2
4,2010-03-10 22:00:00,11.9,59.0,0.7888,1272.0,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8


In [13]:
features.shape

(7111, 12)

In [14]:
features.describe()

,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
count,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000,7111.000000
mean,20.878034,47.561004,1.110309,1091.572100,938.064970,883.903305,1513.238349,998.335565,2.086219,10.237083,204.066784
std,7.937917,17.398731,0.398950,218.537554,281.978988,310.456355,350.180310,381.537695,1.447109,7.694426,193.927723
min,1.300000,8.900000,0.198800,620.300000,364.000000,310.600000,552.900000,242.700000,0.100000,0.100000,1.900000
25%,14.900000,33.700000,0.855900,930.250000,734.900000,681.050000,1320.350000,722.850000,1.000000,4.500000,76.450000
50%,20.700000,47.300000,1.083500,1060.500000,914.200000,827.800000,1513.100000,928.700000,1.700000,8.500000,141.000000
75%,25.800000,60.800000,1.404150,1215.800000,1124.100000,1008.850000,1720.400000,1224.700000,2.800000,14.200000,260.000000
max,46.100000,90.800000,2.231000,2088.300000,2302.600000,2567.400000,2913.800000,2594.600000,12.500000,63.700000,1472.300000


In [15]:
# One-hot encode the data using pandas get_dummies
features = pd.get_dummies(features)
# Display the first 5 rows of the last 12 columns
features.iloc[:,5:].head(5)

,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides,date_time_2010-03-10 18:00:00,date_time_2010-03-10 19:00:00,date_time_2010-03-10 20:00:00,date_time_2010-03-10 21:00:00,...,date_time_2010-12-31 15:00:00,date_time_2010-12-31 16:00:00,date_time_2010-12-31 17:00:00,date_time_2010-12-31 18:00:00,date_time_2010-12-31 19:00:00,date_time_2010-12-31 20:00:00,date_time_2010-12-31 21:00:00,date_time_2010-12-31 22:00:00,date_time_2010-12-31 23:00:00,date_time_2011-01-01 00:00:00
0,1056.0,1742.8,1293.4,2.5,12.0,167.7,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1197.5,1449.9,1010.9,2.1,9.9,98.9,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1060.2,1586.1,1117.0,2.2,9.2,127.1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1102.9,1536.5,1263.2,2.2,9.7,177.2,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1180.9,1415.5,1132.2,1.5,6.4,121.8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# Use numpy to convert to arrays
import numpy as np

# Labels are the values we want to predict
labels = np.array(features['target_carbon_monoxide','target_benzene','target_nitrogen_oxides'])

# Remove the labels from the features
# axis 1 refers to the columns
features= features.drop('target_carbon_monoxide','target_benzene','target_nitrogen_oxides', axis = 1)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

KeyError: ('target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides')